In [1]:
from langchain_core.documents import Document

documents = [
    Document(page_content="LangChain is a framework for developing applications powered by language models.", metadata={"source": "LangChain Docs"}),
    Document(page_content="LangChain enables developers to build applications that can understand and generate natural language.", metadata={"source": "LangChain Overview"}),
]

In [2]:
documents

[Document(metadata={'source': 'LangChain Docs'}, page_content='LangChain is a framework for developing applications powered by language models.'),
 Document(metadata={'source': 'LangChain Overview'}, page_content='LangChain enables developers to build applications that can understand and generate natural language.')]

In [3]:
import os 

from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm =ChatGroq(model="llama-3.1-8b-instant",api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x160bcb770>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1616a8590>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
##Embedding

from langchain_huggingface import HuggingFaceEmbeddings

embeddings=HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

In [6]:
## Vector Stroes --> Convert the doucment into embeddings and than store it in the vector store

from langchain_chroma import Chroma

vectorstore=Chroma.from_documents(documents,embedding=embeddings)

vectorstore

In [10]:
## Async query
await vectorstore.asimilarity_search("developing applications")

vectorstore.similarity_search_with_score("applications")

[(Document(id='8e8dc152-fa75-4a61-a8bf-c44918bb141c', metadata={'source': 'LangChain Overview'}, page_content='LangChain enables developers to build applications that can understand and generate natural language.'),
  1.389512300491333),
 (Document(id='87061a86-83d2-4fec-86a9-2ba0dbc42a65', metadata={'source': 'LangChain Overview'}, page_content='LangChain enables developers to build applications that can understand and generate natural language.'),
  1.389512300491333),
 (Document(id='7e2a615d-d999-4415-810f-77262908d2c2', metadata={'source': 'LangChain Docs'}, page_content='LangChain is a framework for developing applications powered by language models.'),
  1.4404106140136719),
 (Document(id='ecd0dabe-af77-4dae-9030-65f502040c0d', metadata={'source': 'LangChain Docs'}, page_content='LangChain is a framework for developing applications powered by language models.'),
  1.4404106140136719)]

In [ ]:
## Langchain Vector stores  :These object do not subclass runnable therefore ,cannot be integrated directly with our chain.  
# Retrievers : These are runnables , so they implement a set of methods and are designed to be incorporated in chains. Therefore we need to convert vector stores into retreievers

## Convert Langchain vector stores into retreievers

from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever= RunnableLambda(vectorstore.similarity_search).bind(k=1) 
retriever.batch(["application","developers"])



[[Document(id='87061a86-83d2-4fec-86a9-2ba0dbc42a65', metadata={'source': 'LangChain Overview'}, page_content='LangChain enables developers to build applications that can understand and generate natural language.')],
 [Document(id='87061a86-83d2-4fec-86a9-2ba0dbc42a65', metadata={'source': 'LangChain Overview'}, page_content='LangChain enables developers to build applications that can understand and generate natural language.')]]

In [23]:
## Alternate way : to convert vector store into retreiever. It is more efficient

retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)

retriever.batch(["application","developers"])

[[Document(id='87061a86-83d2-4fec-86a9-2ba0dbc42a65', metadata={'source': 'LangChain Overview'}, page_content='LangChain enables developers to build applications that can understand and generate natural language.')],
 [Document(id='87061a86-83d2-4fec-86a9-2ba0dbc42a65', metadata={'source': 'LangChain Overview'}, page_content='LangChain enables developers to build applications that can understand and generate natural language.')]]

In [ ]:
## Integrate the retriever into chain

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
message = """
Answer this question using the provided context only

{question}

Context : 
{context}
 """
prompt=ChatPromptTemplate.from_messages([("human",message)])

rag_chain={"context" : retriever ,"question" : RunnablePassthrough()}|prompt|llm

response=rag_chain.invoke("tell me about application")

print(response)

content='Based on the provided context, it appears that LangChain is an application that enables developers to build applications that can understand and generate natural language.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 109, 'total_tokens': 138, 'completion_time': 0.027276023, 'prompt_time': 0.005925038, 'queue_time': 0.050506012, 'total_time': 0.033201061}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--9b065154-ed8b-4c5a-a4f9-eb331a5fd509-0' usage_metadata={'input_tokens': 109, 'output_tokens': 29, 'total_tokens': 138}
